In [ ]:
import torch
import numpy as np
import pandas as pd
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from collections import OrderedDict

class EvaluationDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        bitboard_str = self.data.loc[idx, "BITBOARD"]
        eval_value = self.data.loc[idx, "EVAL"]

        bin = np.array(list(bitboard_str)).astype(np.float32)
        eval_val = np.array([eval_value]).astype(np.float32)

        return {'binary': bin, 'eval': eval_val}

class EvaluationModel(pl.LightningModule):
    def __init__(self, learning_rate=1e-3, batch_size=512, layer_count=6):
        super().__init__()
        self.batch_size = batch_size
        self.learning_rate = learning_rate

        layers = []
        for i in range(layer_count - 1):
            layers.append((f"linear-{i}", nn.Linear(837, 837)))
            layers.append((f"relu-{i}", nn.ReLU()))
        layers.append((f"linear-{layer_count - 1}", nn.Linear(837, 1)))

        self.seq = nn.Sequential(OrderedDict(layers))

    def forward(self, x):
        return self.seq(x)

    def training_step(self, batch, batch_idx):
        x, y = batch['binary'], batch['eval']
        y_hat = self(x)
        loss = F.l1_loss(y_hat, y)
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.learning_rate)

    def train_dataloader(self):
        dataset = EvaluationDataset(csv_file="/content/data_1.csv")
        return DataLoader(dataset, batch_size=self.batch_size, num_workers=2, pin_memory=True, persistent_workers=True)

config = {"layer_count": 6, "batch_size": 512}

trainer = pl.Trainer(
    devices=1,
    accelerator='cpu',
    precision=16,
    max_epochs=1
)

model = EvaluationModel(
    layer_count=config["layer_count"],
    batch_size=config["batch_size"],
    learning_rate=1e-3
)

trainer.fit(model)

torch.save(model.state_dict(), "/content/chess_model.pth")

In [ ]:
loaded_model = EvaluationModel(layer_count=6, batch_size=512, learning_rate=1e-3)
loaded_model.load_state_dict(torch.load("/content/chess_model.pth"))

additional_dataset = EvaluationDataset(csv_file="/content/data_2.csv")
additional_dataloader = DataLoader(
    additional_dataset,
    batch_size=512,
    num_workers=2,
    pin_memory=True,
    persistent_workers=True
)

trainer = pl.Trainer(
    devices=1,
    accelerator='cpu',
    precision=16,
    max_epochs=1,
)

trainer.fit(loaded_model, additional_dataloader)

torch.save(loaded_model.state_dict(), "/content/chess_model_v02.pth")